# Multi-Tenant RAG Agents with Ollama and Weaviate

This tutorial will demonstrate how to use Weaviate's multi-tenancy to efficiently handle multiple knowledge sources in RAG agents!

# Create Multi-Tenant `KnowledgeSources`

In [15]:
import weaviate

from weaviate.classes.init import Auth

# Connect to Weaviate Cloud
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url="WCS_URL",
    auth_credentials=Auth.api_key("WCS_API"),
)

In [11]:
weaviate_client.collections.delete("KnowledgeSources")

In [12]:
from weaviate.classes.config import Configure
from weaviate.classes.tenants import Tenant

knowledge_sources = weaviate_client.collections.create(
    name="KnowledgeSources",
    multi_tenancy_config=Configure.multi_tenancy(
        enabled=True,
        auto_tenant_creation=True
    )
)

knowledge_sources.tenants.create(
    tenants=[
        Tenant(name="DSPyCode"),
        Tenant(name="DSPyDocs")
    ]
)

In [16]:
tenants = knowledge_sources.tenants.get()

print(tenants)

{'DSPyDocs': Tenant(name='DSPyDocs', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>), 'DSPyCode': Tenant(name='DSPyCode', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}


# Import Data

In [6]:
from datasets import load_dataset

ds = load_dataset("weaviate/chunked_dspy_code")

In [7]:
ds["train"][0]

{'summary': 'The document begins with importing necessary libraries and modules, followed by the definition of the DummyLM class, which is a dummy language model intended for unit testing purposes.',
 'content': 'import random\nimport re\nfrom typing import Union\n\nimport numpy as np\n\nfrom dsp.modules import LM\nfrom dsp.utils.utils import dotdict\n\n\nclass DummyLM(LM):\n    """Dummy language model for unit testing purposes."""',
 'id': '001e04e2-6c05-4ce4-bff6-37bd04c5be26'}

In [13]:
for row in ds["train"]:
    pass

# Build Agent

In [19]:
from typing import Dict

def move_data(collection_use_counter: Dict, move_to_hot_threshold: int) -> None:
    for collection in collection_use_counter.keys():
        if collection_use_counter[collection_name] > move_to_hot_threshold:
            pass
            # move collection to hot
    return